# Trabajo Práctico: reglas de asociación

### Integrantes del grupo: Gonzalez Nehuen, Arja Adel, Madoery Pablo

In [10]:
import numpy as np
from efficient_apriori import apriori
import pandas as pd

In [11]:
movies = pd.read_csv("ml-20m/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
df = pd.read_csv("ml-20m/ratings.csv")
df.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [13]:
# si queremos trabajar con un dataset reducido en numero de users
#ids = list(range(1,11))
#df = df[df.userId.isin(ids)]
#df

,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580
5,1,112,3.5,1094785740
6,1,151,4.0,1094785734
7,1,223,4.0,1112485573
8,1,253,4.0,1112484940
9,1,260,4.0,1112484826


In [14]:
transactions = df.groupby('userId')['movieId'].apply(list).values

In [15]:
itemsets, rules = apriori(transactions, min_support=0.1,  min_confidence=0.7)

## Métricas de reglas de asociación:
### más soporte: la regla se encuentra en más transacciones
### más confianza: mayor probabilidad de que la regla sea cierta para una transacción
### más lift: menor probabilidad de que la regla sea una casualidad
### más convicción: mayor grado de implicación, va de 1 a infinito (si la confianza es 1, la convicción es infinita)

In [16]:
# ordenamos las reglas de mayor a menor lift
rules=sorted(rules, key=lambda rule: rule.lift, reverse=True)
for rule in rules:
  print(rule)

{589} -> {480} (conf: 0.857, supp: 0.667, lift: 1.286, conv: 2.333)
{480} -> {589} (conf: 1.000, supp: 0.667, lift: 1.286, conv: 222222222.222)


In [17]:
# filtramos las reglas 1-1 y las imprimimos ordenadas por lift
filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
  print(rule)

{589} -> {480} (conf: 0.857, supp: 0.667, lift: 1.286, conv: 2.333)
{480} -> {589} (conf: 1.000, supp: 0.667, lift: 1.286, conv: 222222222.222)


In [18]:
# filtramos las reglas 1-1 y las imprimimos ordenadas por lift y decodificadas
filtered_rules = filter(lambda rule: len(rule.lhs) == 1 and len(rule.rhs) == 1, rules)
for rule in sorted(filtered_rules, key=lambda rule: rule.lift, reverse=True):
    m1 = movies[movies.movieId == rule.lhs].title.item()
    m2 = movies[movies.movieId == rule.rhs].title.item()
    print(m1, " -> ", m2)

Terminator 2: Judgment Day (1991)  ->  Jurassic Park (1993)
Jurassic Park (1993)  ->  Terminator 2: Judgment Day (1991)


### Conclusiones